# How Collection Metadata Works in ChromaDB

It works in overwrite mode where the whole metadata for the collection is overwritten.

In [1]:
import chromadb

client = chromadb.Client()

col = client.get_or_create_collection("test",metadata={"test":True})

col.metadata

{'test': True}

In [3]:
col.modify(metadata={"test1":False})
col.metadata

{'test1': False}

# How Collection Metadata Works in ChromaDB

Append only

In [3]:
col.add(ids=["1"],documents=["test"],metadatas=[{"test":True}])

col.get(ids=["1"])

[]


{'ids': ['1'],
 'embeddings': None,
 'metadatas': [{'test': True}],
 'documents': ['test'],
 'uris': None,
 'data': None}

In [6]:
col.upsert(ids=["1"],documents=["test"],metadatas=[{"test1":False}])
col.get(ids=["1"])

{'ids': ['1'],
 'embeddings': None,
 'metadatas': [{'test': True, 'test1': False}],
 'documents': ['test'],
 'uris': None,
 'data': None}

In [7]:
col.update(ids=["1"],documents=["test"],metadatas=[{"test2":False}])
col.get(ids=["1"])

{'ids': ['1'],
 'embeddings': None,
 'metadatas': [{'test': True, 'test1': False, 'test2': False}],
 'documents': ['test'],
 'uris': None,
 'data': None}

In [8]:
col.update(ids=["1"],documents=["test"],metadatas=[{"test2":None}])
col.get(ids=["1"])

ValueError: Expected metadata value to be a str, int, float or bool, got None which is a <class 'NoneType'>

In [1]:
col.upsert(ids=["1"],documents=["test"],metadatas=[None])
col.get(ids=["1"])
# does not work

NameError: name 'col' is not defined

In [1]:

import chromadb

client = chromadb.Client()

col = client.get_or_create_collection("test",metadata={"test":True})
col.add(ids=["1"],documents=["test-meta-none"],metadatas=[{"test":True}])
col.update(ids=["1"],documents=["test"],metadatas=[None])
res = col.get(ids=["1"])
print(res)
assert res["metadatas"][0] is None
# does not work

{'ids': ['1'], 'embeddings': None, 'metadatas': [None], 'documents': ['test'], 'uris': None, 'data': None}


In [2]:
import chromadb

client = chromadb.Client()

col = client.get_or_create_collection("test-partial",metadata={"test":True})
col.add(ids=["1"],documents=["test"],metadatas=[{"test":True,"test1":False}])
print(col.get(ids=["1"]))
col.update(ids=["1"],documents=["test"],metadatas=[{"test1":None}])
res = col.get(ids=["1"])
assert res["metadatas"][0] == {"test":True}

{'ids': ['1'], 'embeddings': None, 'metadatas': [{'test': True, 'test1': False}], 'documents': ['test'], 'uris': None, 'data': None}


In [3]:
import chromadb

client = chromadb.Client()

col = client.get_or_create_collection("test-no-meta",metadata={"test":True})
col.add(ids=["1"],documents=["test-no-meta"],metadatas=[{"test":True,"test1":False}])
print(col.get(ids=["1"]))
col.update(ids=["1"],documents=["test1"]) # this is a bug that removes all the metadata
res = col.get(ids=["1"])
print(res)
assert res["metadatas"][0] == {"test":True,"test1":False}

{'ids': ['1'], 'embeddings': None, 'metadatas': [{'test': True, 'test1': False}], 'documents': ['test-no-meta'], 'uris': None, 'data': None}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'test': True, 'test1': False}], 'documents': ['test1'], 'uris': None, 'data': None}
